In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
!pip install -q peft transformers datasets
!pip install -q bitsandbytes
!pip install -q trl
!pip install sentencepiece

In [3]:
!pip install unsloth_zoo

In [4]:
!pip install --upgrade triton

In [5]:
!pip install --upgrade --no-deps --force-reinstall --no-cache-dir unsloth unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 144.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 285.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 343.0 MB/s eta 0:00:00
  Attempting uninstall: unsloth_zoo
    Found existing installation: unsloth_zoo 2025.7.6
    Uninstalling unsloth_zoo-2025.7.6:
      Successfully uninstalled unsloth_zoo-2025.7.6
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.7.4
    Uninstalling unsloth-2025.7.4:
      Successfully uninstalled unsloth-2025.7.4


In [6]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [8]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.7.4: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    use_rslora = False,
    loftq_config = None
)

Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.7.4 patched 28 layers with 28 QKV layers, 0 O layers and 28 MLP layers.


In [10]:
alpaca_prompt = """در ادامه یک دستورالعمل که یک وظیفه را توصیف می‌کند، همراه با ورودی‌ای که زمینه بیشتری فراهم می‌کند آمده است. پاسخی بنویسید که به‌طور مناسب درخواست را کامل کند.


### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [11]:
EOS_TOKEN = tokenizer.eos_token

In [21]:
def formatting_prompts_func(examples):
  instructions = examples["instruction"]
  inputs = examples["input"]
  outputs = examples["text"]
  texts = []
  for instruction, input, output in zip(instructions, inputs, outputs):
    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    texts.append(text)
  return {"text": texts, }
pass

In [15]:
from datasets import load_dataset
dataset = load_dataset("tarfandoon/alpaca_persian_adabi", split = "train")

In [16]:
dataset

Dataset({
    features: ['text', 'instruction', 'input'],
    num_rows: 1650
})

In [22]:
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/1650 [00:00<?, ? examples/s]

In [23]:
dataset

Dataset({
    features: ['text', 'instruction', 'input'],
    num_rows: 1650
})

In [24]:
dataset['text'][0]

'در ادامه یک دستورالعمل که یک وظیفه را توصیف می\u200cکند، همراه با ورودی\u200cای که زمینه بیشتری فراهم می\u200cکند آمده است. پاسخی بنویسید که به\u200cطور مناسب درخواست را کامل کند.\n\n\n### Instruction:\nمولانا میگه که اگه به هرچی فکر کنیم، می\u200cتونیم بهش برسیم؟\n\n### Input:\nNone\n\n### Response:\nاندیشه\u200cات هر جا رَوَد، وآنگه تو را آن جا کِشَد\n\n• مولانا<|endoftext|>'

In [25]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [27]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "tensorboard",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1650 [00:00<?, ? examples/s]

In [29]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}, Max memory = {max_memory} GB")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4, Max memory = 14.741 GB
7.252 GB of memory reserved.


In [30]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,650 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 37,158,912 of 7,652,775,424 (0.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.197900
2,2.141300
3,2.220600
4,2.001800
5,1.894200
6,1.799200
7,1.739600
8,1.660000
9,1.312800
10,1.287000


In [32]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "شاعر قرن 21 کیست",  # instruction
            "",                  # input
            "",                  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=65, use_cache=True)
tokenizer.batch_decode(outputs)

['در ادامه یک دستورالعمل که یک وظیفه را توصیف می\u200cکند، همراه با ورودی\u200cای که زمینه بیشتری فراهم می\u200cکند آمده است. پاسخی بنویسید که به\u200cطور مناسب درخواست را کامل کند.\n\n\n### Instruction:\nشاعر قرن 21 کیست\n\n### Input:\n\n\n### Response:\nشاعر قرن 21\nمیلاد مولانا<|endoftext|>']

In [35]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "خیلی کوتاه پاسخ بده", # instruction
        "مولانا راجع به بازگشست و سرانجام آدم‌ها پی گفته", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(** inputs, streamer = text_streamer, max_new_tokens = 128)

در ادامه یک دستورالعمل که یک وظیفه را توصیف می‌کند، همراه با ورودی‌ای که زمینه بیشتری فراهم می‌کند آمده است. پاسخی بنویسید که به‌طور مناسب درخواست را کامل کند.


### Instruction:
خیلی کوتاه پاسخ بده

### Input:
مولانا راجع به بازگشست و سرانجام آدم‌ها پی گفته

### Response:
بازگشت و سرانجام آدم‌ها
می‌گویند که هر چه بیشتر
بیشتر از آن که بوده‌ای
بیشتر از آن که بوده‌ای
بیشتر از آن که بوده‌ای
بیشتر از آن که بوده‌ای
بیشتر از آن که بوده‌ای
بیشتر از آن که بوده‌ای



In [36]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')